In [1]:
import os
import sys
module_path = sys.path.append('/home/dpirvu/.local/lib/python3.8/site-packages/')
module_path = sys.path.append('/home/dpirvu/.local/bin/')
for i in sys.path:
    i = i.replace("8", "7")
    if i not in sys.path:
        sys.path.append(i)
        print(i)
import numpy as np
import random
import math
import statistics as stat
import matplotlib
import matplotlib.pyplot as plt
import scipy as scp
import scipy.optimize as sco
import scipy.signal as scs
from scipy import stats
import scipy.special as ssp
import scipy.integrate as sci
from itertools import zip_longest, groupby, cycle
from scipy.ndimage import gaussian_filter1d
from uncertainties import ufloat

/home/dpirvu/.local/lib/python3.7/site-packages/


In [2]:
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

### Simulation Parameters 

In [3]:
nLat = 1024#256#512
nTime = nLat
nSims = 50

phi0 = 1.
m2eff = 1.
lamb = 1.
lenLat = 100.;
nCols = 2

nyq = nLat//2+1
spec = nyq

alpha = 8
dx = lenLat/nLat; print(dx)
dk = 2.*np.pi/lenLat
dt = dx/alpha
dtout = dt*alpha

0.09765625


### Theoretical Field PS 

In [4]:
dynamical_mass_sim_location = lambda nT, nL, tem, phi, lam, m2, gam, sim: '/gpfs/dpirvu/thermal_bubbles/phi4_m2_dynamical_gam{:.4f}'.format(gam)+'_t'+str(nT)+'_x'+str(nL)+'_temp{:.4f}'.format(tem)+'_phi0{:.4f}'.format(phi)+'_lamb{:.4f}'.format(lam)+'_sim'+str(sim)+'_fields.dat'
bare_mass_sim_location = lambda nT, nL, tem, phi, lam, m2, gam, sim: '/gpfs/dpirvu/thermal_bubbles/phi4_m2_stationary_gam{:.4f}'.format(gam)+'_t'+str(nT)+'_x'+str(nL)+'_temp{:.4f}'.format(tem)+'_phi0{:.4f}'.format(phi)+'_lamb{:.4f}'.format(lam)+'_sim'+str(sim)+'_fields.dat'
no_fluct_sim_location = lambda nT, nL, tem, phi, lam, m2, gam, sim: '/gpfs/dpirvu/thermal_bubbles/phi4_m2_static_gam{:.4f}'.format(gam)+'_t'+str(nT)+'_x'+str(nL)+'_temp{:.4f}'.format(tem)+'_phi0{:.4f}'.format(phi)+'_lamb{:.4f}'.format(lam)+'_sim'+str(sim)+'_fields.dat'

#correction includes terms in list sigma0sq_th(list_temp).tolist()
m2_ps_temp_correction_sim_location = lambda nT, nL, tem, phi, lam, m2, gam, sim: '/gpfs/dpirvu/thermal_bubbles/phi4_m2_PS_gam{:.4f}'.format(gam)+'_t'+str(nT)+'_x'+str(nL)+'_temp{:.4f}'.format(tem)+'_phi0{:.4f}'.format(phi)+'_lamb{:.4f}'.format(lam)+'_sim'+str(sim)+'_fields.dat'
#correction includes terms in list [np.mean([np.mean(all_stationary_mass_data[tem][sim,0,0]**2.) for sim in range(nSims)]) for tem in range(len(list_temp))]
m2_eff_late_time_correction_sim_location = lambda nT, nL, tem, phi, lam, m2, gam, sim: '/gpfs/dpirvu/thermal_bubbles/phi4_m2_eff_equil_gam{:.4f}'.format(gam)+'_t'+str(nT)+'_x'+str(nL)+'_temp{:.4f}'.format(tem)+'_phi0{:.4f}'.format(phi)+'_lamb{:.4f}'.format(lam)+'_sim'+str(sim)+'_fields.dat'
# comparison with bare mass intial field
m2_null_sim_location = lambda nT, nL, tem, phi, lam, m2, gam, sim: '/gpfs/dpirvu/thermal_bubbles/phi4_m2_null_gam{:.4f}'.format(gam)+'_t'+str(nT)+'_x'+str(nL)+'_temp{:.4f}'.format(tem)+'_phi0{:.4f}'.format(phi)+'_lamb{:.4f}'.format(lam)+'_sim'+str(sim)+'_fields.dat'

def extract_dynamical_mass_sim(nT, nL, tem, gam, sim):
    data = np.genfromtxt(dynamical_mass_sim_location(nT, nL, tem, phi0, lamb, m2eff, gam, sim))
    return np.asarray([np.reshape(data[:,col], (nT, nL)) for col in range(nCols)])

def extract_stationary_mass_sim(nT, nL, tem, gam, sim):
    data = np.genfromtxt(bare_mass_sim_location(nT, nL, tem, phi0, lamb, m2eff, gam, sim))
    return np.asarray([np.reshape(data[:,col], (nT, nL)) for col in range(nCols)])

def extract_static_sim(nT, nL, tem, gam, sim):
    data = np.genfromtxt(no_fluct_sim_location(nT, nL, tem, phi0, lamb, m2eff, gam, sim))
    return np.asarray([np.reshape(data[:,col], (nT, nL)) for col in range(nCols)])

def extract_m2_ps_temp_correction_sim(nT, nL, tem, gam, sim):
    data = np.genfromtxt(m2_ps_temp_correction_sim_location(nT, nL, tem, phi0, lamb, m2eff, gam, sim))
    return np.asarray([np.reshape(data[:,col], (nT, nL)) for col in range(nCols)])

def extract_m2_eff_late_time_correction_sim(nT, nL, tem, gam, sim):
    data = np.genfromtxt(m2_eff_late_time_correction_sim_location(nT, nL, tem, phi0, lamb, m2eff, gam, sim))
    return np.asarray([np.reshape(data[:,col], (nT, nL)) for col in range(nCols)])

def extract_m2_null_location_sim(nT, nL, tem, gam, sim):
    data = np.genfromtxt(m2_null_sim_location(nT, nL, tem, phi0, lamb, m2eff, gam, sim))
    return np.asarray([np.reshape(data[:,col], (nT, nL)) for col in range(nCols)])

In [5]:
norm = 1./ phi0 / np.sqrt(2.*lenLat)
freqs = np.fft.fftshift(np.fft.fftfreq(nLat, d=dx))*2.*np.pi
omega = freqs**2. + m2eff

spectral_field = lambda tem: np.asarray([norm * np.sqrt(2./(np.exp(omega[k]**0.5/tem) - 1.)) / omega[k]**0.25 if k != nLat//2 else 0. for k in range(nLat)])
sigma0sq_th = lambda tem: np.sum( np.abs(spectral_field(tem))**2. )

V = lambda x: 0.5 * m2eff * x**2. + lamb * x**4. / 24.
th_masses = lambda tem: m2eff + 0.5 * lamb * sigma0sq_th(tem)

### Extract data from files

In [6]:
list_temp = np.arange(1, 16, 1)[:3:]; print(list_temp)
fine_temp_list = np.linspace(min(list_temp), max(list_temp), 100)
filter_size = 0.

[1 2 3]


In [ ]:
all_null_data = np.asarray([[extract_m2_null_location_sim(nTime, nLat, temp, 0., sim) for sim in range(nSims)] for temp in list_temp])
print('Done')

all_ps_mass_data = np.asarray([[extract_m2_eff_late_time_correction_sim(nTime, nLat, temp, 0., sim) for sim in range(nSims)] for temp in list_temp])
print('Done')

all_equil_mass_data = np.asarray([[extract_m2_ps_temp_correction_sim(nTime, nLat, temp, 0., sim) for sim in range(nSims)] for temp in list_temp])
print('Done')

all_data = [all_null_data, all_ps_mass_data, all_equil_mass_data]

In [ ]:
if False:
    sim = random.randint(0, nSims-1)

    fig, ax = plt.subplots(len(all_data), len(list_temp), figsize = (4*len(list_temp)+4, 4*len(all_data)+4))
    for iii in range(len(all_data)):
        for temp in range(len(list_temp)):
            im0 = ax[iii][temp].imshow(all_data[iii][temp][sim][0], aspect='auto', interpolation='none', origin='lower')
            clb = plt.colorbar(im0, ax = ax[iii][temp])
            ax[iii][temp].set_title('T = '+str(list_temp[temp]))
    plt.suptitle('Simulation '+str(sim))
    plt.show()

In [ ]:
if False:
    temp = -1
    fig, ax = plt.subplots(1, len(all_data), figsize = (20, 5))
    for jjj in range(len(all_data)):
        ii = 1
        colors = cycle(('b', 'g', 'orange', 'r', 'm', 'k'))
        for time in np.linspace(0, nTime//10, 5):
            col = next(colors)
            time = int(time)
            ii = ii - 0.17
            PDF1d = []
            for sim in range(nSims):
                for ttime in range(time, time+10):
                    PDF1d += all_data[jjj][temp][sim][0][ttime].tolist()

            PDF1d = np.asarray(PDF1d)
            sigma, mu = np.std(PDF1d), np.mean(PDF1d); print(sigma, mu)
            n, bins, patches = ax[jjj].hist(PDF1d, bins=100, label='t = '+str(time), color=col, alpha=ii, ec='k', histtype='stepfilled')
            x = np.linspace(min(bins), max(bins), 500)
            gaussian = stats.norm.pdf(x, mu, sigma)
            ax[jjj].plot(x, gaussian*(max(n)/max(gaussian)), color=col)

            k2, p = stats.normaltest(PDF1d)
            print("p = {:g}".format(p))
            alpha = 1e-3
            if p < alpha:  # null hypothesis: x comes from a normal distribution
                print("Field is not Gaussian.")
            else:
                print("Field may be Gaussian.")
        ax[jjj].legend()
    plt.show()

In [ ]:
# get mass best fit
omega_fit = lambda k, m: k**2. + m

def get_masses(bool, multiplier, list_temp):  
    masses_null_data, masses_PS_correction_data, masses_late_time_equil_correction_data = [], [], []
    for temp in range(len(list_temp)):
        if bool: fig, ax = plt.subplots(1, 2*len(all_data), figsize = (25, 4))
        for masses, iii in zip([masses_null_data, masses_PS_correction_data, masses_late_time_equil_correction_data], range(len(all_data))):
            PSPECS = np.mean([np.abs(np.fft.fftshift(np.fft.fft2(all_data[iii][temp][sim][0], norm='ortho')))**2. for sim in range(nSims)], axis = 0)
            T, N = PSPECS.shape
            kk = np.fft.fftshift(np.fft.fftfreq(N, d=dx))*2.*np.pi
            oo = np.fft.fftshift(np.fft.fftfreq(T, d=dtout))*2.*np.pi

            maxVal = np.mean(PSPECS) + multiplier * np.std(PSPECS)
            osccoords = np.argwhere(PSPECS > maxVal)

            popt, pcov = sco.curve_fit(omega_fit, kk[osccoords[:,1]], oo[osccoords[:,0]]**2.)
            if popt[-1] >= 0: masses.append(popt[-1])
            else: masses.append('nan')

            if bool:
                lim = 10
                ax[iii*2+0].plot(kk[osccoords[:,1]], oo[osccoords[:,0]], 'o', ms=5, label='freqs')
                ax[iii*2+0].plot(kk, np.sqrt(omega_fit(kk,*popt)), label=f'$m^2=$%5.5f'%popt[-1])
                ax[iii*2+0].set_title(('Bare mass ' if iii == 0 else 'Dynamical mass ')+'T = '+str(list_temp[temp]))
                ax[iii*2+0].set_ylim(0,lim)
                ax[iii*2+0].set_xlim(-lim,lim)
                ax[iii*2+0].axhline(popt[-1], color='darkgray', ls=':', label='best fit')
                ax[iii*2+0].axhline(m2eff, color='darkgray', ls='-.', label='potential mass')
                ax[iii*2+0].legend()

                lim = 3
                ax[iii*2+1].plot(kk, np.sqrt(omega_fit(kk,*popt)), color = 'y', label=f'$m^2=$%5.5f'%popt[-1])
                im = ax[iii*2+1].imshow(PSPECS, aspect='auto', interpolation='none', origin='lower', extent=[kk[0],kk[-1],oo[0],oo[-1]])
                #ax[iii*2+1].imshow(np.amax(PSPECS)/2.*(coords(p3) & cdsnot(p3+0.2)), aspect='auto', interpolation='none', origin='lower', extent=[kk[0],kk[-1],oo[0],oo[-1]])
                clb = plt.colorbar(im, ax = ax[iii*2+1])
                ax[iii*2+1].set_title(('Bare mass ' if iii == 0 else 'Dynamical mass ')+'T = '+str(list_temp[temp]))
                ax[iii*2+1].set_xlabel(r'$\bar{k}$')
                ax[iii*2+1].set_ylabel(r'$\bar{\omega_k}$')
                ax[iii*2+1].set_ylim(0,lim)
                ax[iii*2+1].set_xlim(-lim,lim)
                ax[iii*2+1].axvline(0, color='white', ls=':')
                ax[iii*2+1].axhline(0, color='white', ls=':')
                ax[iii*2+1].legend()
        if bool: plt.show()
    return np.asarray(masses_null_data), np.asarray(masses_PS_correction_data), np.asarray(masses_late_time_equil_correction_data)

In [ ]:
if True:
    multiplier = 1
    list_temperatures_temporary = list_temp

    mass_from_data = get_masses(True, multiplier, list_temp)
    list_titles = [r'$m^2(T)$ from $V(\phi; m_{bare}^2)$', r'$m^2(T)$ from $V(\phi; m^2 + \Delta m_{PS}^2)$, PS correction (or RMS at t=0)', r'$m^2(T)$ from $V(\phi; m^2 + \Delta m_{equil}^2)$, RMS at late time equilibrium']

    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    for iii in range(len(all_data)):
        plt.plot(list_temp, mass_from_data[iii], 'o', ls='-', label=list_titles[iii])

    plt.axhline(m2eff, color='darkgray', ls='--', label='potential mass')
    plt.xlabel(r'$T$')
    plt.ylabel(r'$m^2(T)$')
    plt.legend()
    plt.show()

### Get profile of omega ^2

In [ ]:
def get_pspec(iii, tem, kkmax_to_average):
    pspecs = np.mean([np.abs(np.fft.fftshift(np.fft.fft2(all_data[iii][tem][sim][0], norm='ortho')))**2. for sim in range(nSims)], axis = 0)
    T, N = pspecs.shape
    kk = np.fft.fftshift(np.fft.fftfreq(N, d=dx))*2.*np.pi
    oo = np.fft.fftshift(np.fft.fftfreq(T, d=dtout))*2.*np.pi

    pick_freq_window = (kk>0)&(kk<kkmax_to_average)#; print('freqs in window = ', kk[pick_freq_window])
    pick_omega = np.sort(oo[oo>0])[0]#; print(f'$\omega = $', pick_omega)
    return np.mean(pspecs[:, pick_freq_window], axis=1)[oo>0], pspecs[oo==pick_omega, kk>0], oo[oo>0], kk[kk>0]

if True:
    kkmax_to_average = 0.5
    masses_from_dispersion_relation = get_masses(False, multiplier, list_temp)

    for tem in range(len(list_temp)):
        marks = cycle(('o', '*', 'v'))
        lsss = cycle(('-', ':', '--'))
        colors = cycle(('b', 'g', 'orange', 'r', 'm', 'k'))
        fig, ax = plt.subplots(1, 2, figsize = (25, 5))
        for iii in range(len(all_data)):
            col = next(colors)

            window_in_k, fix_omega, oo, kk = get_pspec(iii, tem, kkmax_to_average)   

            ax[0].plot(oo, window_in_k, label=['Null', 'PS', 'equil'][iii]+f', $T$ = '+str(list_temp[tem]), color=col)
            ax[0].axvline(np.sqrt(masses_from_dispersion_relation[iii][tem]), color=col, ls='--', label='mass from best fit')

            ax[1].plot(kk, fix_omega, label=f'$T$ = '+str(list_temp[tem]), color=col, marker=next(marks), ls=next(lsss))

        ax[1].set_yscale('log')
        [i.set_xlim(0, 3) for i in ax]
        [ax[i].set_xlabel((f'$\omega_k$' if i == 0 else f'$k$')) for i in range(len(ax))]
        [i.set_ylabel(f'$|FFT(\phi_x)|^2$') for i in ax]
        [i.legend() for i in ax]
        plt.show()